<a href="https://colab.research.google.com/github/JAYANTHNITW/RAGS_LLMS/blob/main/falcon_finetuning_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
pip install -q datasets

In [3]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import  AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig



In [4]:
from transformers import  AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [5]:
from peft import (
    PeftConfig,
    LoraConfig,
    PeftModel,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
)

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
pip install

In [2]:
model_id ="EleutherAI/pythia-70m"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

NameError: ignored

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0},trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
import transformers
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np

In [ ]:
# load dataset from huggingface datasets
ds = load_dataset("OpenAssistant/oasst1")
# lets convert the train dataset to a pandas df
df = ds["train"].to_pandas()
new_df = df["text"]
new_df.to_csv("new_df.csv")
new_data = load_dataset("csv",data_files={"train":"new_df.csv"})

In [ ]:
df

In [ ]:


tokenizer.pad_token = tokenizer.eos_token

data = new_data.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True,), batched=True)
data



In [ ]:
import os
os.environ["NEPTUNE_API_TOKEN"]=""
os.environ["NEPTUNE_PROJECT"]=""

In [ ]:
training_args = transformers.TrainingArguments(
   per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
   lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
    fp16=True,
    save_total_limit=4,
    logging_steps=25,
    output_dir="outputs",
    optim="paged_adamw_8bit"
)
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
#trainer.save_model("./Quotes Model")
model.save_pretrained("open_assistant_model")
trainer.save_model("open_assistant_trainer")

In [ ]:
prompt = """<human>: My name is Daniel. Write a short email to my closest friends inviting them to come to my home on Friday for a dinner party, I will make the food but tell them to BYOB.
<bot>:"""

In [ ]:

tokenizer.pad_token = tokenizer.eos_token

batch = tokenizer(
    prompt,
    padding=True,
    truncation=True,
    return_tensors='pt'
)
batch = batch.to('cuda:0')


In [ ]:
with torch.cuda.amp.autocast():
    output_tokens = model.generate(
        input_ids = batch.input_ids,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.7,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )

In [ ]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# Inspect message response in the outputs
print(generated_text.split("<human>: ")[1].split("<bot>: ")[-1])

In [ ]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# Inspect message response in the outputs
print(generated_text.split("<human>: ")[1].split("<bot>: ")[-1])

In [ ]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# Inspect message response in the outputs
print(generated_text.split("<human>: ")[1].split("<bot>: ")[-1])

In [ ]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# Inspect message response in the outputs
print(generated_text.split("<human>: ")[1].split("<bot>: ")[-1])

In [ ]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# Inspect message response in the outputs
print(generated_text.split("<human>: ")[1].split("<bot>: ")[-1])

In [ ]:
generated_text

In [ ]:

prompt = "Hi I'm a Facebook ad copywriter and I'd be happy to help you with your Facebook ad copy. Here's an example of a Facebook ad copy that I've written for a client: Are you tired of losing money in the stock market?"
prompt = "say me a joke"
tokenizer.pad_token = tokenizer.eos_token
batch = tokenizer(
    prompt,
    padding=True,
    truncation=True,
    return_tensors='pt'
)
batch = batch.to('cuda:0')
with torch.cuda.amp.autocast():
    output_tokens = model.generate(
        input_ids = batch.input_ids,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.7,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

In [ ]:
%%timeit
with torch.cuda.amp.autocast():
    output_tokens = model.generate(
        input_ids = batch.input_ids,
        max_new_tokens=50,
        temperature=0.7,
        top_p=0.7,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)